### Lab 3.1: Basic Neural Network in PyTorch - Solution

Let's create a linear classifier one more time, but using PyTorch's automatic differentiation and optimization algorithms.  Then you will extend the perceptron into a multi-layer perceptron (MLP).

In [1]:
import numpy as np
import torch

We need to explicitly tell PyTorch when creating a tensor that we are interested in later computing its gradient

In [2]:
a = torch.tensor(5.,requires_grad=True)
a

tensor(5., requires_grad=True)

In [3]:
b = torch.tensor(6.,requires_grad=True)
c = 2*a+3*b
c

tensor(28., grad_fn=<AddBackward0>)

To extract the gradients, we first need to call `backward()`.

In [4]:
c.backward()

Now to get the gradient of any variable with respect to `c`, we simply access the `grad` attribute of that variable.

In [5]:
a.grad

tensor(2.)

In [6]:
b.grad

tensor(3.)

Let's load and format the Palmer penguins dataset for multi-class classification.

In [7]:
from palmerpenguins import load_penguins
from matplotlib import pyplot as plt

In [8]:
df = load_penguins()

# drop rows with missing values
df.dropna(inplace=True)

# get two features
X = df[['flipper_length_mm','bill_length_mm']].values

# convert species labels to integers
y = df['species'].map({'Adelie':0,'Chinstrap':1,'Gentoo':2}).values

To make the learning algorithm work more smoothly, we we will subtract the mean of each feature.

Here `np.mean` calculates a mean, and `axis=0` tells NumPy to calculate the mean over the rows (calculate the mean of each column).

In [9]:
X -= np.mean(X,axis=0)

Now we will convert our `X` and `y` arrays to torch Tensors.

In [10]:
X = torch.tensor(X).float()
y = torch.tensor(y).long()

In [11]:
from torch import nn

The `torch.nn.Sequential` class creates a feed-forward network from a list of `nn.Module` objects.  Here we provide a single `nn.Linear` class which performs an affine transformation ($Wx+b$) so that we will have a linear classifier.

In [12]:
linear_model = torch.nn.Sequential(
    torch.nn.Linear(2,3), # two inputs, three outputs
)

Now we create a cross-entropy loss function object and a stochastic gradient descent (SGD) optimizer.

In [13]:
loss_fn = torch.nn.CrossEntropyLoss()

In [14]:
lr = 1e-2
opt = torch.optim.SGD(linear_model.parameters(), lr=lr)

Finally we can iteratively optimize the model.

In [15]:
epochs = 100
for epoch in range(epochs):
    opt.zero_grad() # zero out the gradients

    z = linear_model(X) # compute z values
    loss = loss_fn(z,y) # compute loss

    loss.backward() # compute gradients

    opt.step() # apply gradients

    print(f'epoch {epoch}: loss is {loss.item()}')

epoch 0: loss is 5.746509075164795
epoch 1: loss is 3.662224769592285
epoch 2: loss is 2.4052345752716064
epoch 3: loss is 1.9485793113708496
epoch 4: loss is 1.722406029701233
epoch 5: loss is 1.5718508958816528
epoch 6: loss is 1.458114743232727
epoch 7: loss is 1.3650604486465454
epoch 8: loss is 1.2849527597427368
epoch 9: loss is 1.2136406898498535
epoch 10: loss is 1.1486711502075195
epoch 11: loss is 1.0885024070739746
epoch 12: loss is 1.0321266651153564
epoch 13: loss is 0.9788690209388733
epoch 14: loss is 0.92827308177948
epoch 15: loss is 0.8800314664840698
epoch 16: loss is 0.8339424133300781
epoch 17: loss is 0.7898805141448975
epoch 18: loss is 0.7477768659591675
epoch 19: loss is 0.7076043486595154
epoch 20: loss is 0.6693655252456665
epoch 21: loss is 0.6330825090408325
epoch 22: loss is 0.5987884998321533
epoch 23: loss is 0.5665190815925598
epoch 24: loss is 0.5363045334815979
epoch 25: loss is 0.5081632137298584
epoch 26: loss is 0.48209652304649353
epoch 27: loss i

### Exercises

Extend the above code to implement an MLP with a single hidden layer of size 100.

Write code to compute the accuracy of each model.

Can you get the MLP to outperform the linear model?

In [17]:
mlp = torch.nn.Sequential(
    torch.nn.Linear(2,100),
    torch.nn.ReLU(),
    torch.nn.Linear(100,3),
)

opt = torch.optim.SGD(mlp.parameters(), lr=lr)

for epoch in range(epochs):
    opt.zero_grad() # zero out the gradients

    z = mlp(X) # compute z values
    loss = loss_fn(z,y) # compute loss

    loss.backward() # compute gradients

    opt.step() # apply gradients

    print(f'epoch {epoch}: loss is {loss.item()}')

epoch 0: loss is 1.5085662603378296
epoch 1: loss is 0.5871738791465759
epoch 2: loss is 0.4000960886478424
epoch 3: loss is 0.322466105222702
epoch 4: loss is 0.27651047706604004
epoch 5: loss is 0.2495923489332199
epoch 6: loss is 0.23229750990867615
epoch 7: loss is 0.22056759893894196
epoch 8: loss is 0.21197642385959625
epoch 9: loss is 0.2052653729915619
epoch 10: loss is 0.19975292682647705
epoch 11: loss is 0.19506588578224182
epoch 12: loss is 0.1909896433353424
epoch 13: loss is 0.18738973140716553
epoch 14: loss is 0.18417416512966156
epoch 15: loss is 0.18127582967281342
epoch 16: loss is 0.17864347994327545
epoch 17: loss is 0.1762370616197586
epoch 18: loss is 0.1740245372056961
epoch 19: loss is 0.17197997868061066
epoch 20: loss is 0.1700822412967682
epoch 21: loss is 0.16831357777118683
epoch 22: loss is 0.16665948927402496
epoch 23: loss is 0.16510747373104095
epoch 24: loss is 0.1636471152305603
epoch 25: loss is 0.16226951777935028
epoch 26: loss is 0.16096688807010

In [18]:
# Compute the accuracy of each model

def accuracy(y_pred,y_true):
    return (y_pred == y_true).float().mean()

y_pred = linear_model(X).argmax(dim=1)
acc = accuracy(y_pred,y)
print(f'linear model accuracy: {acc.item()}')

y_pred = mlp(X).argmax(dim=1)
acc = accuracy(y_pred,y)
print(f'mlp accuracy: {acc.item()}')

linear model accuracy: 0.9489489197731018
mlp accuracy: 0.9519519805908203


MLP has outperformed the linear model with it having an accuracy of 0.952 while the linear model accuracy is 0.949.